In [3]:
import hub

# Load the datasets
ds_train = hub.load(r"E:\Projets\Datasets - Pytorch\Food\hub\train", read_only=True)
ds_val = hub.load(r"E:\Projets\Datasets - Pytorch\Food\hub\val", read_only=True)
ds_test = hub.load(r"E:\Projets\Datasets - Pytorch\Food\hub\test", read_only=True)

# Check the keys in the dataset


E:\Projets\Datasets - Pytorch\Food\hub\train loaded successfully.

E:\Projets\Datasets - Pytorch\Food\hub\val loaded successfully.

E:\Projets\Datasets - Pytorch\Food\hub\test loaded successfully.



In [4]:
import torch
from torch.utils.data import Dataset, DataLoader, Subset
from torchvision import transforms
import hub
import numpy as np

# Define the transformation for the images (you can modify this)
data_transforms = transforms.Compose([
    transforms.Resize(size=(224, 224)),
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Custom Dataset class to wrap the hub dataset
from PIL import Image

class HubDataset(Dataset):
    def __init__(self, hub_dataset, transform=None):
        self.dataset = hub_dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset['images'])  # Assuming 'images' is the key for images

    def __getitem__(self, idx):
        # Get image (assuming 'images' is the correct key)
        image = self.dataset['images'][idx].numpy()

        # Convert the NumPy array to a PIL image
        image = Image.fromarray(image)

        # Get category index from 'categories'
        category_index = self.dataset['categories'][idx].numpy()[0]  # Adjust based on label format
        
        # Apply transformation to the image
        if self.transform:
            image = self.transform(image)

        # Convert category index to tensor
        label = torch.tensor(category_index, dtype=torch.long)
        
        return image, label



# Create PyTorch datasets
train_dataset = HubDataset(ds_train, transform=data_transforms)
val_dataset = HubDataset(ds_val, transform=data_transforms)
test_dataset = HubDataset(ds_test, transform=data_transforms)


def get_subset_indices(dataset, percentage=0.1):
    total_samples = len(dataset)
    indices = list(range(total_samples))
    np.random.shuffle(indices)  # Shuffle the indices
    subset_size = int(percentage * total_samples)
    return indices[:subset_size]

# Step 2: Get 20% of the data
train_indices = get_subset_indices(train_dataset, percentage=0.2)
val_indices = get_subset_indices(val_dataset, percentage=0.2)
test_indices = get_subset_indices(test_dataset, percentage=0.2)


train_subset = Subset(train_dataset, train_indices)
val_subset = Subset(val_dataset, val_indices)
test_subset = Subset(test_dataset, test_indices)

train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_subset, batch_size=32, shuffle=False)

# Example: Check the size of the dataloaders
print(f"Training samples: {len(train_loader.dataset)}")
print(f"Validation samples: {len(val_loader.dataset)}")
print(f"Test samples: {len(test_loader.dataset)}")


Training samples: 7992
Validation samples: 200
Test samples: 600


In [5]:
import torch
import torch.nn as nn

Alexmodel = model = torch.hub.load('pytorch/vision:v0.10.0', 'googlenet', pretrained=True)

def weights_init(m):
    if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
    elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)

Alexmodel.apply(weights_init)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(Alexmodel.parameters() , lr= 0.0001 , weight_decay=1e-4)  

Using cache found in C:\Users\Shaikh Abdul Rafay/.cache\torch\hub\pytorch_vision_v0.10.0
c:\Users\Shaikh Abdul Rafay\anaconda3\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Shaikh Abdul Rafay\anaconda3\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:
# Move model to device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
Alexmodel.to(device)

# Early stopping and epochs
eps = 3
train_losses = []
val_losses = []
train_accuracies = []
val_accuracies = []

for epoch in range(eps):
    train_correct = 0
    train_total = 0
    running_loss = 0

    Alexmodel.train()

    for image, categories in train_loader:
        image, categories = image.to(device), categories.to(device)
        optimizer.zero_grad()
        outputs = Alexmodel(image)

        loss = criterion(outputs, categories)
        l1_norm = sum(p.abs().sum() for p in Alexmodel.parameters())
        loss += 0.01 * l1_norm

        _, predicted = torch.max(outputs.data, 1)
        train_total += categories.size(0)
        train_correct += (predicted == categories).sum().item()

        loss.backward()
        running_loss += loss.item()
        optimizer.step()

    avg_training_loss = running_loss / len(train_loader)
    train_accuracy = (train_correct / train_total)
    train_losses.append(avg_training_loss)
    train_accuracies.append(train_accuracy)

    val_correct = 0
    val_total = 0
    val_running_loss = 0

    Alexmodel.eval()
    with torch.no_grad():
        for image, categories in val_loader:
            image, categories = image.to(device), categories.to(device)
            outputs = Alexmodel(image)
            loss = criterion(outputs, categories)
            val_running_loss += loss.item()  

            _, predicted = torch.max(outputs.data, 1)
            val_total += categories.size(0)
            val_correct += (predicted == categories).sum().item()

    avg_val_loss = val_running_loss / len(val_loader)  
    val_accuracy = (val_correct / val_total)

    val_accuracies.append(val_accuracy)
    val_losses.append(avg_val_loss)

    # Print results
    print(f'Epoch: {epoch}, Train Loss: {avg_training_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')


Epoch: 0, Train Loss: 1886.0566, Train Accuracy: 0.00%, Val Loss: 6.8747, Val Accuracy: 0.00%
Epoch: 1, Train Loss: 950.3702, Train Accuracy: 0.02%, Val Loss: 6.7233, Val Accuracy: 0.07%
Epoch: 2, Train Loss: 488.7217, Train Accuracy: 0.07%, Val Loss: 6.6837, Val Accuracy: 0.07%


In [13]:
print(ds_test.tensors)


{'images': Tensor(key='images'), 'images_meta': Tensor(key='images_meta')}
